# Check Yourself.
This notebook is intended to explore your ability to connect to a `Fabric SQL Endpoint`

Required to have the `ODBC Driver 18 for SQL Server` installed & use `az login`

## Python Notebook to Connect to Fabric SQL Endpoint

- This notebook will help to troubleshoot Fabric SQL Endpoint connectivity that will be used for building our Agents via Semantic Kerenel and through the Agent Service with tools excluding the Fabric Data Agent.  This is not the ideal path, but if the Fabric Data Agent is not able to provide the quality of answers it does when called directly though Fabric, this will help to guide our Architecture choices.

In [69]:
import pyodbc
import os
from dotenv import load_dotenv  
import pandas as pd
from sqlalchemy import create_engine
import struct

load_dotenv(override=True) 

True

In [70]:
import os
import pyodbc
from sqlalchemy import create_engine
from azure import identity

# Check for available drivers
available_drivers = pyodbc.drivers()
preferred_driver = ["ODBC Driver 18 for SQL Server"]


# # Select the first available preferred driver
selected_driver = next((driver for driver in preferred_driver if driver in available_drivers), None)

if not selected_driver:
    raise Exception(f"None of the preferred drivers ({preferred_driver}) are installed. Available drivers: {available_drivers}")
else:
    print(f"Using driver: {selected_driver}")

Using driver: ODBC Driver 18 for SQL Server


## Confirm you can access Fabric

use: az login

In [71]:
load_dotenv(override=True) 
warehouse = os.environ.get("FABRIC_WAREHOUSE")
print(f"Using warehouse: {warehouse}")

Using warehouse: wwilakehouse_01


In [72]:
import urllib  
import os  
import struct  
from azure import identity  
from sqlalchemy import create_engine, text  
  
fabric_connection_string = os.environ.get("FABRIC_CONNECTION_STRING")  
driver = "ODBC Driver 18 for SQL Server"  
  
odbc_str = (  
    f"Driver={{{driver}}};"  
    f"Server=tcp:{fabric_connection_string},1433;"  
    "Encrypt=yes;"  
    "TrustServerCertificate=no;"  
    "Connection Timeout=30;"  
)  
  
quoted_odbc_str = urllib.parse.quote_plus(odbc_str)  
  
def get_access_token():  
    credential = identity.DefaultAzureCredential(  
        exclude_interactive_browser_credential=False  
    )  
    token = credential.get_token("https://database.windows.net/.default").token  
    token_bytes = token.encode("UTF-16-LE")  
    return token_bytes  
  
def get_engine():  
    token_bytes = get_access_token()  
    token_struct = struct.pack("<I{}s".format(len(token_bytes)), len(token_bytes), token_bytes)  
    SQL_COPT_SS_ACCESS_TOKEN = 1256  # Defined by Microsoft  
    engine = create_engine(  
        f"mssql+pyodbc:///?odbc_connect={quoted_odbc_str}",  
        connect_args={  
            "attrs_before": {SQL_COPT_SS_ACCESS_TOKEN: token_struct}  
        }  
    )  
    return engine  


## Get Tables

In [73]:
warehouse = os.environ.get("FABRIC_WAREHOUSE", "NONE")
query = """SELECT *   
FROM [{warehouse}].INFORMATION_SCHEMA.TABLES  
WHERE TABLE_TYPE = 'BASE TABLE';  """.format(warehouse=warehouse)
df = pd.read_sql(query, con=get_engine())
print(df.to_html(index=False, border=0, justify='center', classes='table table-striped table-bordered'))

<table class="dataframe table table-striped table-bordered">
  <thead>
    <tr style="text-align: center;">
      <th>TABLE_CATALOG</th>
      <th>TABLE_SCHEMA</th>
      <th>TABLE_NAME</th>
      <th>TABLE_TYPE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>wwilakehouse_01</td>
      <td>dbo</td>
      <td>dimension_customer</td>
      <td>BASE TABLE</td>
    </tr>
  </tbody>
</table>


### Get Table Schemas

In [80]:
load_dotenv(override=True) 
warehouse = os.environ.get("FABRIC_WAREHOUSE", "NONE")
table = os.environ.get("FABRIC_TEST_TABLE", "NONE")
print(f"Using warehouse: {warehouse}, table: {table}")
query = """
SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, IS_NULLABLE
                FROM [{warehouse}].INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_NAME = '{table_name}'
                """.format(warehouse=warehouse, table_name=table)

print(f"Query: {query}")


df = pd.read_sql(query, con=get_engine())
print(df.to_html(index=False, border=0, justify='center', classes='table table-striped table-bordered'))

Using warehouse: wwilakehouse_01, table: dimension_customer
Query: 
SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, IS_NULLABLE
                FROM [wwilakehouse_01].INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_NAME = 'dimension_customer'
                
<table class="dataframe table table-striped table-bordered">
  <thead>
    <tr style="text-align: center;">
      <th>COLUMN_NAME</th>
      <th>DATA_TYPE</th>
      <th>CHARACTER_MAXIMUM_LENGTH</th>
      <th>IS_NULLABLE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>CustomerKey</td>
      <td>int</td>
      <td>NaN</td>
      <td>YES</td>
    </tr>
    <tr>
      <td>WWICustomerID</td>
      <td>int</td>
      <td>NaN</td>
      <td>YES</td>
    </tr>
    <tr>
      <td>Customer</td>
      <td>varchar</td>
      <td>8000.0</td>
      <td>YES</td>
    </tr>
    <tr>
      <td>BillToCustomer</td>
      <td>varchar</td>
      <td>8000.0</td>
      <td>YES</td>
    </tr>
    <tr>
      <td>Category</td>
     